<a href="https://colab.research.google.com/github/sora-ix9/learning_on_Pytorch/blob/main/word_embeddings_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings: Encoding Lexical Semantics

This notebook contains codes followingly practicing from [the Pytorch tutorial on Word Embeddings: Encoding Lexical Semantics](https://docs.pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

# Set-up

Check Pytorch version

In [ ]:
!pip list | grep torch

torch                                    2.8.0+cu126
torchao                                  0.10.0
torchaudio                               2.8.0+cu126
torchdata                                0.11.0
torchsummary                             1.5.1
torchtune                                0.6.1
torchvision                              0.23.0+cu126


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 2.) Word Embeddings in Pytorch

In [ ]:
word_2_ind = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5) # The two specified parameter are the row and column of embedding matrix, where row corresponds to vocabulary size (i.e. |V|).
lookup_tensor = torch.tensor([word_2_ind['hello']], dtype=torch.long)
hello_embed = embeds(lookup_tensor)

print('embeds =', embeds)
print()
print('lookup_tensor =', lookup_tensor)
print()
print('lookup_tensor.shape =', lookup_tensor.shape)
print()
print('hello_embed =', hello_embed)
print()
print('hello_embed.shape =', hello_embed.shape)

embeds = Embedding(2, 5)

lookup_tensor = tensor([0])

lookup_tensor.shape = torch.Size([1])

hello_embed = tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)

hello_embed.shape = torch.Size([1, 5])


# 3.) An Example: N-Gram Language Modeling

The below cell contains my arbitrary defined code retuning ngrams for better interpretable code.

In [ ]:
def get_ngrams(CONTEXT_SIZE, test_sentence, verbose=False):
  ngrams = []
  for i in range(CONTEXT_SIZE, len(test_sentence)):

    context = []
    for j in range(CONTEXT_SIZE):
      context.append(test_sentence[i-j-1])

    tup = (context, test_sentence[i]) # The first element is a list containing context for the target word, and the second element is the target word.
    ngrams.append(tup)

  return ngrams

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# The five below lines are my arbitrary defined code retuning ngrams for better interpretable code.
print('The two lines below are result from my arbitrary defined code retuning ngrams for better interpretable code.')
print('len(test_sentence) =', len(test_sentence))
ngrams = get_ngrams(CONTEXT_SIZE, test_sentence)
print('ngrams =', ngrams)
print()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

vocab = set(test_sentence)
word_2_ind = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGramLanguageModeler, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # This embeddings is equivalent to input layer in MLP.
    self.linear_layer1 = nn.Linear(context_size * embedding_dim, 128) # This linear_layer1 is equivalent to fully connected layer in MLP.
    self.linear_layer2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))

    layer1_output = F.relu(self.linear_layer1(embeds))
    layer2_ouput = self.linear_layer2(layer1_output)
    log_probs = F.log_softmax(layer2_ouput, dim=1)

    return log_probs

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):

  total_loss = 0
  for context, target in ngrams:

    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
    # into integer indices and wrap them in tensors)
    context_ind = torch.tensor([word_2_ind[w] for w in context], dtype=torch.long)

    # Step 2. Recall that torch *accumulates* gradients. Before passing in a
    # new instance, you need to zero out the gradients from the old
    # instance
    model.zero_grad()

    # Step 3. Run the forward pass, getting log probabilities over next words
    log_probs = model(context_ind)

    # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
    target_tensor = torch.tensor([word_2_ind[target]], dtype=torch.long)
    '''
    print('context_ind =', context_ind)
    print('word_2_ind[target] =', word_2_ind[target])
    print(target_tensor.shape)
    print()
    '''
    loss = loss_function(log_probs, target_tensor)

    # Step 5. Do the backward pass and update the gradient
    loss.backward()
    optimizer.step()

    # Get the Python number from a 1-element Tensor by calling tensor.item()
    total_loss += loss.item()

  losses.append(total_loss)

# Getting the embedding of a particular word (e.g. "beauty")
beauty_embed = model.embeddings.weight[word_2_ind['beauty']]

print('The lines below are result the tutorial code.')
print('len(test_sentence) =', len(test_sentence))
print('ngrams =', ngrams)
print()
print('len(vocab) =', len(vocab))
print()
print('losses =', losses)
print()
print('beauty_embed =', beauty_embed)

The two lines below are result from my arbitrary defined code retuning ngrams for better interpretable code.
len(test_sentence) = 115
ngrams = [(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege'), (['besiege', 'shall'], 'thy'), (['thy', 'besiege'], 'brow,'), (['brow,', 'thy'], 'And'), (['And', 'brow,'], 'dig'), (['dig', 'And'], 'deep'), (['deep', 'dig'], 'trenches'), (['trenches', 'deep'], 'in'), (['in', 'trenches'], 'thy'), (['thy', 'in'], "beauty's"), (["beauty's", 'thy'], 'field,'), (['field,', "beauty's"], 'Thy'), (['Thy', 'field,'], "youth's"), (["youth's", 'Thy'], 'proud'), (['proud', "youth's"], 'livery'), (['livery', 'proud'], 'so'), (['so', 'livery'], 'gazed'), (['gazed', 'so'], 'on'), (['on', 'gazed'], 'now,'), (['now,', 'on'], 'Will'), (['Will', 'now,'], 'be'), (['be', 'Will'], 'a'), (['a', 'be'], "totter'd"), (["totter'd", 'a'], 'weed'), (['weed', "totter'd"], 'of'), (['of', 'weed'], 'small'), (['small', 'of'], 'worth'), (['wor

In [ ]:
model

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear_layer1): Linear(in_features=20, out_features=128, bias=True)
  (linear_layer2): Linear(in_features=128, out_features=97, bias=True)
)

In [ ]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

# 4.) Exercise: Computing Word Embeddings: Continuous Bag-of-Words

In [ ]:
CONTEXT_SIZE = 2 # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_2_ind = {w: i for i, w in enumerate(vocab)}
# data = []

def get_data(CONTEXT_SIZE, raw_text):
  data = []

  for i in range(CONTEXT_SIZE, len(raw_text) - 2):

    right_context = [raw_text[i-j-1] for j in range(CONTEXT_SIZE)]
    left_context = [raw_text[i+j+1] for j in range(CONTEXT_SIZE)]
    context = right_context + left_context
    target = raw_text[i]
    data.append((context, target))

  return data

data = get_data(CONTEXT_SIZE, raw_text)

class CBOW(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(CBOW, self).__init__()
    self.embedding_layer = nn.Embedding(vocab_size, embedding_dim) # Embedding layer

    self.linear_layer1 = nn.Linear(context_size * 2 * embedding_dim, 128)
    self.activation_func1 = nn.ReLU()

    self.linear_layer2 = nn.Linear(128, vocab_size)
    self.activation_func2 = nn.LogSoftmax(dim=1)

  def forward(self, inputs):
    embeds = self.embedding_layer(inputs).view((1, -1))

    layer1_output = self.linear_layer1(embeds)
    layer1_output = self.activation_func1(layer1_output) # ReLU function

    layer2_output = self.linear_layer2(layer1_output)

    log_probs = self.activation_func2(layer2_output) # Log softmax function

    return log_probs

def fit_CBOW(CBOW_model, data, epoch):
  losses = []
  loss_func = nn.NLLLoss()
  optimizer = optim.SGD(CBOW_model.parameters(), lr=0.001)

  for epoch in range(epoch):
    total_loss = 0

    for context, target in data:
      # Step 1. Prepare the inputs to be passed to the model (i.e. turn the words into integer indices and wrap them in tensors)

      #   Define a long tensor of context indices
      context_ind = [word_2_ind[w] for w in context]
      context_ind = torch.tensor(context_ind, dtype=torch.long)

      #   Define a long tensor of target word's index
      target_ind = torch.tensor([word_2_ind[target]], dtype=torch.long)

      # Step 2. Recall that torch *accumulates* gradients. Before passing in a
      # new instance, you need to zero out the gradients from the old instance
      CBOW_model.zero_grad()

      # Step 3. Run the forward pass, getting log probabilities over next words
      log_probs = CBOW_model(context_ind)

      # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
      loss = loss_func(log_probs, target_ind)

      # Step 5. Do the backward pass and update the gradient
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

    losses.append(total_loss)

  return CBOW_model, losses

CBOW_model = CBOW(vocab_size=vocab_size, embedding_dim=10, context_size=CONTEXT_SIZE)
CBOW_model, losses = fit_CBOW(CBOW_model=CBOW_model, data=data, epoch=10)

computer_ind = word_2_ind['computer']
computer_embedding = CBOW_model.embedding_layer.weight[computer_ind]

print('vocab_size =', vocab_size)
print('data =', data)
print('len(data) =', len(data))
print()
print('losses =', losses)
print()
print('computer_embedding =', computer_embedding)

vocab_size = 49
data = [(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea'), (['idea', 'the', 'a', 'computational'], 'of'), (['of', 'idea', 'computational', 'process.'], 'a'), (['a', 'of', 'process.', 'Computational'], 'computational'), (['computational', 'a', 'Computational', 'processes'], 'process.'), (['process.', 'computational', 'processes', 'are'], 'Computational'), (['Computational', 'process.', 'are', 'abstract'], 'processes'), (['processes', 'Computational', 'abstract', 'beings'], 'are'), (['are', 'processes', 'beings', 'that'], 'abstract'), (['abstract', 'are', 'that', 'inhabit'], 'beings'), (['beings', 'abstract', 'inhabit', 'computers.'], 'that'), (['that', 'beings', 'computers.', 'As'], 'inhabit'), (['inhabit', 'that', 'As', 'they'], 'computers.'), (['computers.', 'inhabit', 'they', 'evolve,'], 'As'), (['As', 'computers.', 

Test CBOW

In [ ]:
CBOW_model

CBOW(
  (embedding_layer): Embedding(49, 10)
  (linear_layer1): Linear(in_features=40, out_features=128, bias=True)
  (activation_func1): ReLU()
  (linear_layer2): Linear(in_features=128, out_features=49, bias=True)
  (activation_func2): LogSoftmax(dim=1)
)

In [ ]:
test_context = ['Computational', 'processes', 'abstract', 'beings']
test_context_ind = [word_2_ind[w] for w in test_context]
test_context_ind = torch.tensor(test_context_ind, dtype=torch.long)
ind_2_word = {word_2_ind[w]: w for w in word_2_ind}

pred = CBOW_model(test_context_ind)
most_pred_ind = torch.argmax(prob_pred[0]).item()
most_pred_word = ind_2_word[most_pred_ind]

'''
print(prob_pred)
print(most_prob_pred)
'''

print(f"Raw text = {' '.join(raw_text)}")
print()
print('Test context =', test_context)
print()
print('Prediction =', most_pred_word)

Raw text = We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Test context = ['Computational', 'processes', 'abstract', 'beings']

Prediction = processes.
